In [166]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset ,DataLoader
import itertools as it

### Input / output format
Input is in de vorm van twee vectors die waardes 1 en 0 bevatten (waar of onwaar).\
De waardes van de **eerste** vector moeten worden gelezen alsof er conjuncten tussen staan, een voorbeeld is de vector (0,1) die kan worden gelezen als: niet A en B.\
De waardes van de **tweede** vector moeten worden gelezen alsof er disjuncten tussen staan, verder zijn er altijd exact twee true values in de vector.\
Deze vectoren worden vergeleken (bijvoorbeeld (0,0,1) en (0,1,1) is True) en de output zal true of false zijn.\
De hoeveelheid mogelijke combinaties zijn 2^n * (n!/(2!(n-2)!)), voor n = 3 is dit 24.\
\
Alle combinaties waar de eerste vector gelijk is aan de tweede vector worden verwijderd.\
\
In deze poging ga ik de tien keer regel gebruiken, dit houd in dat voor elk mogelijk input-output paar er tien kopieëren.

In [167]:
#generate input vectors
def generate_input_vectors(n):
    fist_vec = list(it.product([0, 1], repeat=n))
    second_vec = []
    for i in fist_vec:
        if i.count(1) == 2:
            second_vec.append(i)
    total_vec = list(it.product(fist_vec, second_vec))
    for i in total_vec:
        if i[0] == i[1]:
            total_vec.remove(i)
    return total_vec
def generate_val(n):
    fist_vec = list(it.product([0, 1], repeat=n))
    second_vec = []
    for i in fist_vec:
        if i.count(1) == 2:
            second_vec.append(i)
    total_vec = list(it.product(fist_vec, second_vec))
    return total_vec
#generate true output
def generate_true(x):
    out = []
    for pair in x:
        for i in range(len(pair[0])):
            if pair[0][i] == 1 and pair[1][i] == 1:
                out.append(((pair[0],pair[1]),True))
                break
            elif i == len(pair[0]) - 1:
                out.append(((pair[0],pair[1]),False))
    return out
print(generate_true(generate_input_vectors(3)))

[(((0, 0, 0), (0, 1, 1)), False), (((0, 0, 0), (1, 0, 1)), False), (((0, 0, 0), (1, 1, 0)), False), (((0, 0, 1), (0, 1, 1)), True), (((0, 0, 1), (1, 0, 1)), True), (((0, 0, 1), (1, 1, 0)), False), (((0, 1, 0), (0, 1, 1)), True), (((0, 1, 0), (1, 0, 1)), False), (((0, 1, 0), (1, 1, 0)), True), (((0, 1, 1), (1, 0, 1)), True), (((0, 1, 1), (1, 1, 0)), True), (((1, 0, 0), (0, 1, 1)), False), (((1, 0, 0), (1, 0, 1)), True), (((1, 0, 0), (1, 1, 0)), True), (((1, 0, 1), (0, 1, 1)), True), (((1, 0, 1), (1, 1, 0)), True), (((1, 1, 0), (0, 1, 1)), True), (((1, 1, 0), (1, 0, 1)), True), (((1, 1, 1), (0, 1, 1)), True), (((1, 1, 1), (1, 0, 1)), True), (((1, 1, 1), (1, 1, 0)), True)]


In [168]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.X = [torch.tensor(i[0]) for i in data]
        self.y = torch.tensor([i[1] for i in data], dtype=torch.bool)
        self.data = list(zip(self.X, self.y))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [169]:
data_train = generate_true(generate_input_vectors(5))
dataset_train = CustomDataset(data_train)
data_val = generate_true(generate_val(5))
dataset_val = CustomDataset(data_val)
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=4, shuffle=True)

In [170]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(10, 5)
        self.fc2 = nn.Linear(5, 2)

    def forward(self, x):
        x = x.view(-1, 10)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
model = NeuralNet()

In [171]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [172]:
epochs = 500
for epoch in range(epochs):
    model.train()
    for inputs, labels in dataloader_train:
        optimizer.zero_grad()
        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
    print(f'Epoch: [{epoch+1}/{epochs}], Loss: {loss.item()}')

Epoch: [1/500], Loss: 0.9158954620361328
Epoch: [2/500], Loss: 0.769905686378479
Epoch: [3/500], Loss: 0.24967069923877716
Epoch: [4/500], Loss: 0.2581193447113037
Epoch: [5/500], Loss: 0.7415211200714111
Epoch: [6/500], Loss: 0.25861889123916626
Epoch: [7/500], Loss: 0.22418934106826782
Epoch: [8/500], Loss: 0.2063371241092682
Epoch: [9/500], Loss: 0.7607000470161438
Epoch: [10/500], Loss: 0.18082782626152039
Epoch: [11/500], Loss: 0.18745160102844238
Epoch: [12/500], Loss: 0.13137900829315186
Epoch: [13/500], Loss: 0.1553906947374344
Epoch: [14/500], Loss: 0.5288422107696533
Epoch: [15/500], Loss: 0.4234210252761841
Epoch: [16/500], Loss: 0.9661648869514465
Epoch: [17/500], Loss: 0.6987106800079346
Epoch: [18/500], Loss: 0.4150815010070801
Epoch: [19/500], Loss: 0.33340224623680115
Epoch: [20/500], Loss: 0.8342223763465881
Epoch: [21/500], Loss: 0.031053654849529266
Epoch: [22/500], Loss: 0.6232700347900391
Epoch: [23/500], Loss: 0.2265084981918335
Epoch: [24/500], Loss: 0.8287472128

In [173]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in dataloader_val:
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 100.0%


In [174]:
# inputs, labels = next(iter(dataloader_val))
# outputs = model(inputs.float())
# _, predicted = torch.max(outputs.data, 1)

# fig, ax = plt.subplots(1, 4, figsize=(12, 3))
# for i, axes in enumerate(ax):
#     axes.imshow(inputs[i].view(2, 5))
#     axes.set_title(f'Predicted: {predicted[i]}, Actual: {labels[i]}')
#     axes.axis('off')
# plt.show()